# SBA 빅데이터 사이언스 실무연계 프로젝트 🏫

## Machine Learning sevice on DSVM ☁️ - 세 번째
---
### 03. Cifar10 Image Classification Model Deploy
* [Azure Machine Learning를 사용하여 모델 배포](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/how-to-deploy-and-where/?WT.mc_id=AI-MVP-5003262)  
* [Azure Machine Learning에서 기존 모델 사용
](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/how-to-deploy-existing-model/?WT.mc_id=AI-MVP-5003262)

---

## Part1. 배포 환경 설정

1. Workspace 연결
2. datastore 객체 가져오기
3. 생성된 모델확인

### 1-1. Workspace 연결

In [ ]:
import azureml
from azureml.core import Workspace

# Azure core SDK version 확인
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

### 1-2. datastore 객체 연결

In [ ]:
ds = ws.get_default_datastore()
print(ds)

### 1-3. 모델 객체 연결

In [ ]:
from azureml.core.model import Model

model = Model(ws, 'keras_cifar10_gpu')

In [ ]:
print(model.id)

---
## Part2. 이미지 등록 및 배포하기

1. 모델 동작 코드 확인(score.py)
2. Container 이미지 환경 파일 생성(myenv.yml)
3. Azure Container Instance 생성
4. Container Image 설정
5. 이미지 Build 및 Instance Deploy

**참고자료**  
[Azure Container Instance에서 이미지 분류 모델 배포](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/tutorial-deploy-models-with-aml/?WT.mc_id=AI-MVP-5003262)

### 2-1. score.py 내용 확인

In [ ]:
# 모델 동작 코드 확인
import os

with open('./score.py', 'r') as f:
    print(f.read())

### 2-2. Container 이미지 환경 파일 생성

In [ ]:
# Container 이미지 동작 환경 설정 파일을 생성
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('tensorflow')
cd.add_conda_package('keras')
cd.add_conda_package('scikit-learn')
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

### 2-3. Azure Container Instance 배포 내용 설정

In [ ]:
# 인스턴스 동작 환경을 설정
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=2, 
                                               tags={'name':'cifar10', 'framework': 'Keras'},
                                               description='Keras MLP on cifar10')

### 2-4. Container Image 설정

In [ ]:
# Container 이미지 동작 환경을 저장
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

### 2-5. 이미지 Build 및 Instance Deploy

In [ ]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(workspace=ws,
                                       name='keras-cifar10-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

---
## Part3. 배포된 Instance 사용하기

1. 배포된 Instnace 의 접속 url 확인
2. 배포된 Instnace 사용하기

### 3-1. 배포된 Instance 의 접속 url 확인

In [ ]:
print(service.scoring_uri)

### 3-2. 배포된 Instance 사용하기

In [ ]:
# 이미지와 라벨 데이터를 가져오기
from sklearn.model_selection import train_test_split
import numpy as np

cifar10_label = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

images = np.load('./data/images.npy')
label = np.load('./data/label.npy')

X_train, X_test, y_train, y_test = train_test_split(images, label, test_size = 0.2)

print("Total Images:", images.shape)
print("Total Label:", label.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

In [ ]:
import json

# 테스트 셋 중 랜덤으로 10개 선별
n = 10
sample_indices = np.random.permutation(X_test.shape[0])[0:n]

test_samples = json.dumps({"data": X_test[sample_indices].tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# 실제 배포된 서비스로 예측 동작 확인
result = service.run(input_data=test_samples)

In [ ]:
# 예측 결과 시각화
%matplotlib inline

import matplotlib.pyplot as plt

i = 0
plt.figure(figsize = (20, 1))

for s in sample_indices:
    plt.subplot(1, n, i + 1)
    plt.axhline('')
    plt.axvline('')
    
    # use different color for misclassified sample
    font_color = 'red' if y_test[s] != result[i] else 'black'
    clr_map = plt.cm.gray if y_test[s] != result[i] else plt.cm.Greys
    
    plt.text(x=10, y=-10, s=cifar10_label[i], fontsize=18, color=font_color)
    plt.imshow(X_test[s], cmap=clr_map)
    
    i = i + 1
plt.show()

In [ ]:
# local 이미지 예측
from PIL import Image
local_image = 'local_image.jpg'
image = Image.open(local_image)

# resize
resized_image = image.resize((32, 32))
images = np.array(resized_image)

data = np.zeros((1, 32, 32, 3), dtype=np.uint8)
data[0] = images

local_test_sample = json.dumps({"data": data.tolist()})
local_test_sample = bytes(local_test_sample, encoding='utf8')

result = service.run(input_data= local_test_sample)[0]
print('Result is:', cifar10_label[result])

---
수고하셨습니다 🕊